## Hidden Markdov Model and Ngram(Graded): Named Entity Relationship Dataset

Welcome to your programming assignment on HMM and NGram! you will build HMM to classify tagged words.


Note: This notebook would take more than 8 mins to run depending on GPU Availablity


It is a common task that involves identifying and categorizing  named entities. The project include on simpler task of sentence likelihood prediction, that involves in determining the probability of a given sentence being valid based on a Hidden Markov Model(HMM). Let's also implement the Viterbi algorithm to determine the best probable path and compare it with the hmmlearn implementation. Additionally, formulated an HMM and completed the forward and backward function in the Baum-Welch algorithm to improve the HMM model. 

### Dataset Description

#### Content
The dataset with 1M x 4 dimensions contains columns = ['# Sentence', 'Word', 'POS', 'Tag'] and is grouped by #Sentence.

#### Columns
- **Word**: This column contains English dictionary words from the sentence it is taken from.
- **POS**: Parts of speech tag

#### Instructions
- Do not modify any of the codes.
- Only write code when prompted. For example in some sections you will find the following,
```
# YOUR CODE GOES HERE
# YOUR CODE ENDS HERE
# TODO
```

Only modify those sections of the code.

In [ ]:
import hmmlearn
from hmmlearn import hmm

### Imports

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt  # show graph
import random

#some other libraries
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from typing import List

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score

from Assignment_helpers import *

## Load the data



In [ ]:
data = pd.read_csv("data/NER dataset.csv", encoding='latin1')

The dataset contains a total of 47960 sentences, with 1,048,575 words. The entities includes 9 named entity types including person names, locations, organizations, dates, times, percentages and others.



## Task: Data pre-processing
- Fill missing values in the dataset using forward fill method.


In [ ]:
data = 0 #TODO: Fill missing values in the dataset
data = data.rename(columns={'Sentence #': 'sentence'})
data.head(5)

Task: This pre-processing (lowercase any words, remove stop words, replace numbers/names by a unique NUM/NAME token, etc.) you should list of english stop words here in the pipeline.


In [ ]:
def pre_processing(text_column):
    # lowercase all text in the column
    text_column = text_column.str.lower()

    # replacing numbers with NUM token
    text_column = text_column.str.replace(r'\d+', 'NUM')

    # removing stopwords
    stop_words = 0 # TODO: Get the list of stopwords from the nltk library in English
    text_column = text_column.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    return text_column


Task: Create a new dataset the unique words and the unique tags in the dataset, by calling `pre_process()` to keep both versions and compare the effect of your pre-processing.

In [ ]:
data_pre_precessed = 0 # TODO: Preprocess the data

The pre-processing step is performed to clean and transform the data in a way that is more suitable for further analysis. The pre-processing functions performed includes, converting all the words in the 'Word' column to lowercase and this is done to standardize the capitalization of the words and ensure that they are all in the same case. Then replaced all the numeric values in the 'Word' column with the token 'NUM'. This is done to avoid overfitting of the model to specific numerical values that may not be relevant to the task. Then removed the stop words, which are common words that don't convey any significant meaning in the context of the text. The number of unique tags and unique words in the original dataset was 42 and 29764 respectively. After pre-processing and removing empty/null rows, the number of unique words is reduced to 24031.

In [ ]:
data_pre_precessed.head(20)

Task: There's lot of missing data in this preprocessed frame let's handle it.
Assign the pre-processed 'Word' column to the data_processed dataframe.

Task: Remove rows where the 'Word' column is empty.

In [ ]:
#creating new dataframe with preprocessed word as a column
data_processed = data
data_processed['Word'] = 0 # TODO: Add the preprocessed data to the new dataframe

#removing the rows where word is empty
data_processed = 0 # TODO: Remove the rows where the word is empty

In [ ]:
data_processed.head(20)

In [ ]:
tags = list(set(data.POS.values))  # Unique POS tags in the dataset
words = list(set(data.Word.values))  # Unique words in the dataset
len(tags), len(words)

We should have 42 tags and 29764 words

Task: Get the unique words from the preprocessed data

In [ ]:
words1 = 0  # TODO: from preprocessed data get unique words in the dataset
len(words1)

We should have length of 29763 

### We have 42 different tags and 29,764 different words, so the HMM that we construct will have the following properties
- The hidden states of the this HMM will correspond to the tags, so we will have 42 hidden states.
- The Observations for this HMM will correspond to the sentences and their words.

#### Before constructing the HMM, we will split the data into train and test.

In [ ]:
y = data.POS
X = data.drop('POS', axis=1)

gs = GroupShuffleSplit(n_splits=2, test_size=.33, random_state=42)
train_ix, test_ix = next(gs.split(X, y, groups=data['sentence']))

data_train = data.loc[train_ix]
data_test = data.loc[test_ix]

In [ ]:
data_train.head(5)

In [ ]:
data_test.head(5)

In [ ]:
#using preprocessed data 

y1 = data_processed.POS
X1 = data_processed.drop('POS', axis=1)
data_processed.reset_index(drop=True, inplace=True)
gs = GroupShuffleSplit(n_splits=2, test_size=.33, random_state=42)
train_ix1, test_ix1 = next(gs.split(X1, y1, groups=data_processed['sentence']))

data_train1 = data_processed.loc[train_ix1]
data_test1 = data_processed.loc[test_ix1]

In [ ]:
data_train1.head()

In [ ]:
data_test1.head()

Now lets encode the POS and Words to be used to generate the HMM.

In [ ]:
dfupdate = data_train.sample(frac=.15, replace=False, random_state=42)
dfupdate.Word = 'UNKNOWN'
data_train.update(dfupdate)
words = list(set(data_train.Word.values))
# Convert words and tags into numbers
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}
id2tag = {i: t for i, t in enumerate(tags)}
len(tags), len(words)

In lecture you might have heard that the Hidden Markov Models can be learned by using the Baum-Welch algorithm by just using the observations.
Although we can learn the Hidden States (tags) using Baum-Welch algorithm,We cannot map them back the states (words) to the tag. So for this exercise we will skip using the BW algorithm and directly create the HMM.

For creating the HMM we should build the following three parameters. 
- `startprob_`
- `transmat_`
- `emissionprob_`

To construct the above mentioned paramters let's first create some useful matrices that will assist us in creating the above three parameters

In [ ]:
count_tags = dict(data_train.POS.value_counts())  # Total number of POS tags in the dataset
# Now let's create the tags to words count
count_tags_to_words = data_train.groupby(['POS']).apply(
    lambda grp: grp.groupby('Word')['POS'].count().to_dict()).to_dict()
# We shall also collect the counts for the first tags in the sentence
count_init_tags = dict(data_train.groupby('sentence').first().POS.value_counts())

# Create a mapping that stores the frequency of transitions in tags to it's next tags
count_tags_to_next_tags = np.zeros((len(tags), len(tags)), dtype=int)
sentences = list(data_train.sentence)
pos = list(data_train.POS)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        prevtagid = tag2id[pos[i - 1]]
        nexttagid = tag2id[pos[i]]
        count_tags_to_next_tags[prevtagid][nexttagid] += 1

Now Let's build the parameter matrices 

In [ ]:
startprob = np.zeros((len(tags),))
"""
Initializes and populates the start probability, transition probability, and emission probability matrices for a Hidden Markov Model (HMM).

Variables:
    startprob (numpy.ndarray): A 1D array of size (number of tags,) initialized to zeros, representing the initial state probabilities.
    transmat (numpy.ndarray): A 2D array of size (number of tags, number of tags) initialized to zeros, representing the state transition probabilities.
    emissionprob (numpy.ndarray): A 2D array of size (number of tags, number of words) initialized to zeros, representing the emission probabilities.
    num_sentences (int): The total number of sentences, calculated as the sum of initial tag counts.
    sum_tags_to_next_tags (numpy.ndarray): A 1D array representing the sum of transitions from each tag to the next tags.

Loop Variables:
    tag (str): The current tag in the iteration.
    tagid (int): The corresponding ID of the current tag.
    floatCountTag (float): The count of the current tag converted to float.
    word (str): The current word in the iteration.
    wordid (int): The corresponding ID of the current word.
    tag2 (str): The next tag in the nested iteration.
    tagid2 (int): The corresponding ID of the next tag.

Operations:
    - Calculates the initial state probabilities for each tag.
    - Calculates the emission probabilities for each tag-word pair.
    - Calculates the transition probabilities for each tag-tag pair.
"""
transmat = np.zeros((len(tags), len(tags)))
emissionprob = np.zeros((len(tags), len(words)))
num_sentences = sum(count_init_tags.values())
sum_tags_to_next_tags = np.sum(count_tags_to_next_tags, axis=1)
for tag, tagid in tqdm(tag2id.items(), position=0, leave=True):
    floatCountTag = float(count_tags.get(tag, 0))
    startprob[tagid] = count_init_tags.get(tag, 0) / num_sentences
    for word, wordid in word2id.items():
        emissionprob[tagid][wordid] = count_tags_to_words.get(tag, {}).get(word, 0) / floatCountTag
    for tag2, tagid2 in tag2id.items():
        transmat[tagid][tagid2] = count_tags_to_next_tags[tagid][tagid2] / sum_tags_to_next_tags[tagid]

#### Task: Similar to how we built the hidden state transition probability matrix as shown above, you will build the transition probability between the words. With this matrix write a function that can calculate the log likelihood given a sentence.

In [ ]:
#To create word transition matrix

#first step is to count the number of times each word appears in the dataset
count_words = {}
for word in data_train.Word.values:
    count_words[word] = count_words.get(word, 0) + 1

# then count the number of times a word appears after another word
count_word_transitions = {}
for sentence in data_train.groupby('sentence'):
    words = sentence[1]['Word'].values
    for i in range(len(words) - 1):
        w1, w2 = words[i], words[i+1]
        if w1 not in count_word_transitions:
            count_word_transitions[w1] = {}
        count_word_transitions[w1][w2] = count_word_transitions[w1].get(w2, 0) + 1

# convert the counts to probabilities
word_transition_matrix = 0 # TODO: Create a matrix of zeros with the shape of the number of unique words (Hint: Make sure to use the word2id dictionary)
sum_words_to_next_words = 0 # TODO: Calculate the sum of words to next words
for w1, w1id in word2id.items():
    for w2, w2id in word2id.items():
        word_transition_matrix[w1id][w2id] = count_word_transitions.get(w1, {}).get(w2, 0) / sum_words_to_next_words
print(word_transition_matrix.shape)

We should see the matrix shape of 23608, 23608

In [ ]:
def calculate_log_likelihood(sentence: List[str], word_transition_matrix) -> float:
    """
    Given a sentence and word_transition_matrix, returns the log-likelihood of the sentence.
    """
    # converting the sentence to a list of word IDs
    sentence_ids = [word2id.get(w, word2id['UNKNOWN']) for w in sentence]

    # calculating the log-likelihood using the word transition matrix
    log_likelihood = np.log(word_transition_matrix[sentence_ids[0]][sentence_ids[1]])
    for i in range(1, len(sentence_ids) - 1):
        log_likelihood += np.log(word_transition_matrix[sentence_ids[i]][sentence_ids[i+1]] + 1e-10)
    return log_likelihood

In [ ]:
calculate_log_likelihood([0], word_transition_matrix) # TODO: Replace the 0 with a test sentence of tags

In this task, a word transition probability matrix is built and a function that can calculate the log likelihood given a sentence. The transition probability matrix was created by first counting the number of times each word appeared in the training dataset and then counting the number of times a word appeared after another word. These counts were then converted to probabilities to create the transition probability matrix.

Then wrote a function to calculate the log-likelihood of a given sentence using the word transition probabilities. The sentence was first converted into a list of word IDs, and then the log-likelihood was calculated using the word transition probabilities. 

#### Task: Now we will continue to constructing the HMM.

We will use the hmmlearn implementation to initialize the HMM Model

In [ ]:
model = hmm.MultinomialHMM(n_components=0, algorithm='', random_state=42) # TODO: Create a Multinomial HMM model with the number of components equal to the number of unique tags using the vertibi algorithm.
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

#### Before using the HMM to predict the POS tags, we have to fix the training set as some of the words and tags in the test data might not appear in the training data so we collect this data to use it later.

In [ ]:
data_test.loc[~data_test['Word'].isin(words), 'Word'] = 'UNKNOWN'
word_test = list(data_test.Word)
samples = []
for i, val in enumerate(word_test):
    samples.append([word2id[val]])

    
lengths = []
count = 0
sentences = list(data_test.sentence)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        count += 1
    elif i > 0:
        lengths.append(count)
        count = 1
    else:
        count = 1

Now that we have the HMM ready lets predict the best path from them.

In [ ]:
pos_predict = model.predict(samples, lengths)
pos_predict

The hmmlearn predict function will give the best probable path for the given sentence using the Viterbi algorithm.

### Task: Using the model parameters (startprob_, transmat_, emissionprob_) write the viterbi algorithm to calculate the best probable path and compare it with the hmmlearn implementation.

Now before using these matrices 

viterbi algorithm to find the best probable path using the model parameters startprob, transmat, emissionprob. The algorithm takes in the initial probabilities, transition probabilities, emission probabilities and a list of obeservations and returns an array of the indices of the best hidden states.

In [ ]:

def viterbi(pi: np.array, a: np.array, b: np.array, obs: List) -> np.array:
    """
     Write the viterbi algorithm from scratch to find the best probable path
     attr:
       pi: initial probabilities
       a: transition probabilities
       b: emission probabilities
       obs: list of observations
     return:
       array of the indices of the best hidden states
    """
    # state space cardinality
    K = a.shape[0]

    # observation sequence length
    T = len(obs)

    # initializing the tracking tables from first observation
    delta = np.zeros((T, K))
    psi = np.zeros((T, K))
    delta[0] = 0 # TODO: Initialize the delta table with the initial probabilities and the first observation

    # iterating throught the observations updating the tracking tables
    for t in range(1, T):
        for j in range(K):
            delta[t, j] = np.max(delta[t-1] * a[:, j] * b[j, obs[t]]) 
            psi[t, j] = 0 # TODO: Update the delta and psi tables using argmax function

    # build the output, optimal model trajectory
    x = np.zeros(T, dtype=int)
    x[T-1] = np.argmax(delta[T-1])
    for t in range(T-2, -1, -1):
        x[t] = psi[t+1, x[t+1]]

    return x

### Task: Let's try to form our own HMM
In this task you will try to formulate your own HMM. Image a toy example that you think that closely relates to a Hidden Markov Model.

Steps:
 1. Define your hidden states
 2. Define your observable states
 3. Randomly generate your observations

Below is an example to demonstrate:

-In this toy HMM example, we have two hidden states 'healthy' and 'sick' these states relate to the state of a pet. In this example we cannot exactly know the situation of the pet if it is 'healthy' or 'sick'

-The observable states in this formulation is the what our pet is doing, whether it is sleeping, eating or pooping. We ideally want to determine if the pet is sick or not using these observable states


```python
hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observations = []
for i in range(100):
  observations.append(random.choice(observable_states))
```

In [ ]:
import random

hidden_states = 0 # TODO: Create a list of all the weather seasons
observable_states = 0 #TODO: Create a list of all possible of how the weather feels like
observations =  []

for i in range(40):
  obs_index = random.randint(0, len(observable_states)-1) # random index corresponding to the observable state
  observations.append(obs_index) # then adding the index to the observations list

In [ ]:
hidden_state_sequence = viterbi(startprob, transmat, emissionprob, observations)

print("Observations:", observations)
print("Viterbi sequence:", hidden_state_sequence)

Even tough we have generated the data randomly, for the learning purposes, let's try to learn an HMM from this data. let's use Baum-Welch algorithm

### TASK: let's try with observable map

In [ ]:
import random

hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observable_map = 0 # TODO: Create a dictionary mapping the observable states to integers
observations = []
for i in range(40):
    observations.append(observable_map[random.choice(observable_states)])

A, B = baum_welch(observations=observations, observations_vocab=np.array(list(observable_map.values())),
                  n_hidden_states=2)


Implemented the Baum-Welch algorithm for estimating the HMM parameters by the forward, backward, and E-step functions to compute the gamma and sigma probabilities. These probabilities are used to estimate the HMM parameters.

In [ ]:
hidden_state_sequence = viterbi(startprob, transmat, emissionprob, observations)

print("Observations:", observations)
print("Viterbi sequence:", hidden_state_sequence)